
# Etapa 02 – Engenharia de Atributos (Feature Engineering)

Nesta etapa realizamos o processo de engenharia de atributos para enriquecer o dataset com novas variáveis derivadas de forma lógica e estatística.

## Objetivos
- Criar variáveis sintéticas que reforcem padrões de risco de desligamento.
- Codificar variáveis categóricas para uso em modelos.
- Escalonar variáveis numéricas para padronização.
- Preparar os dados para treinamento e teste.

## Novas Features Criadas

| Nome | Descrição |
|------|-----------|
| `SatisfacaoMedia` | Média entre satisfação no trabalho, ambiente e relacionamento. |
| `FaixaEtaria` | Categorização da idade em faixas: `<30`, `30-45`, `>45`. |
| `DistanteDoTrabalho` | Binária: se distância > 20km. |
| `ViajaMuito` | Binária: se viagem a trabalho é frequente. |
| `FormacaoSuperiorOuMais` | Binária: escolaridade >= superior. |
| `RiscoHorasExtras` | Binária: horas extras com baixo equilíbrio vida-trabalho. |
| `TempoSemPromocao` | Copia do tempo desde a última promoção. |
| `ExperienciaPorEmpresa` | Anos de experiência dividido por nº de empresas anteriores + 1. |
| `SalarioAjustado` | Salário dividido pelo nível do cargo. |
| `EstabilidadeNaEmpresa` | Anos na empresa dividido por anos de experiência. |

## Pré-processamento

- **Codificação:** Utilizamos `LabelEncoder` para colunas categóricas.
- **Escalonamento:** Utilizamos `StandardScaler` para colunas numéricas.
- **Split:** Separação entre treino e teste em 70/30, estratificado.

Essas transformações melhoram a capacidade dos modelos de capturar padrões relevantes.

In [12]:
# Feature Engineering Unificada: Geração de Novas Features + Pré-processamento
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import sys
import os
sys.path.append(os.path.abspath("../"))
from scripts.func import gerar_base_rh_analitico

# 1. Gerar a base original
df = gerar_base_rh_analitico(qtd_amostras=100_000, semente=42)

# 2. Criar novas features com base na base original (sem escalonar ainda)
df['SatisfacaoMedia'] = df[['SatisfacaoTrabalho', 'SatisfacaoAmbiente', 'SatisfacaoRelacionamento']].mean(axis=1)
df['FaixaEtaria'] = pd.cut(df['Idade'], bins=[17, 29, 45, 66], labels=['<30', '30-45', '>45'])
df['DistanteDoTrabalho'] = (df['DistanciaCasa'] > 20).astype(int)
df['ViajaMuito'] = (df['ViagemTrabalho'] == 'Frequente').astype(int)
df['FormacaoSuperiorOuMais'] = (df['Escolaridade'] >= 4).astype(int)
df['RiscoHorasExtras'] = ((df['HorasExtras'] == 'Sim') & (df['EquilibrioVida'] <= 2)).astype(int)
df['TempoSemPromocao'] = df['AnosUltimaPromocao']
df['ExperienciaPorEmpresa'] = df['AnosExperiencia'] / (df['EmpresasAnteriores'] + 1)
df['SalarioAjustado'] = df['SalarioMensal'] / df['NivelCargo'].replace(0, 1)
df['EstabilidadeNaEmpresa'] = df['AnosEmpresa'] / df['AnosExperiencia'].replace(0, 1)

# 3. Codificar variáveis categóricas
df_encoded = df.copy()
label_cols = ['Genero', 'AreaFormacao', 'Setor', 'Cargo', 'ViagemTrabalho', 'HorasExtras', 'EstadoCivil', 'Desligamento', 'FaixaEtaria']
label_encoders = {}
for col in label_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    label_encoders[col] = le

# 4. Escalonamento de variáveis numéricas
num_cols = ['Idade', 'AnosExperiencia', 'AnosEmpresa', 'AnosCargoAtual', 'AnosUltimaPromocao', 'AnosGestorAtual',
            'EmpresasAnteriores', 'SalarioMensal', 'DistanciaCasa', 'SatisfacaoMedia',
            'ExperienciaPorEmpresa', 'SalarioAjustado', 'EstabilidadeNaEmpresa']

scaler = StandardScaler()
df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])

# 5. Separação treino e teste
X = df_encoded.drop('Desligamento', axis=1)
y = df_encoded['Desligamento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# 6. Amostra
print("\n🎯 Target balance:")
print(y.value_counts(normalize=True) * 100)

print("\n📊 Shape dos conjuntos:")
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)

# Exibe amostra
print(df_encoded[['SatisfacaoMedia', 'FaixaEtaria', 'DistanteDoTrabalho', 'ViajaMuito',
                         'FormacaoSuperiorOuMais', 'RiscoHorasExtras', 'TempoSemPromocao',
                         'ExperienciaPorEmpresa', 'SalarioAjustado', 'EstabilidadeNaEmpresa']].head())


🎯 Target balance:
Desligamento
0    79.367
1    20.633
Name: proportion, dtype: float64

📊 Shape dos conjuntos:
X_train: (70000, 32)
X_test : (30000, 32)
   SatisfacaoMedia  FaixaEtaria  DistanteDoTrabalho  ViajaMuito  \
0        -1.063939            0                   0           1   
1         0.197340            0                   0           0   
2         1.458618            0                   1           0   
3        -1.063939            2                   0           1   
4         1.458618            0                   0           0   

   FormacaoSuperiorOuMais  RiscoHorasExtras  TempoSemPromocao  \
0                       1                 0                 6   
1                       0                 0                 0   
2                       0                 0                 3   
3                       1                 0                 3   
4                       0                 0                 6   

   ExperienciaPorEmpresa  SalarioAjustado  Estabili